In [174]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
from torchinfo import summary

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

In [175]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
path = "./Dataset/"
mmscaler = MinMaxScaler(feature_range=(-1, 1), copy=True)

#training set
train_ax = mmscaler.fit_transform(np.loadtxt(path + "train/Inertial Signals/total_acc_x_train.txt"))
train_ay = mmscaler.fit_transform(np.loadtxt(path + "train/Inertial Signals/total_acc_y_train.txt"))
train_az = mmscaler.fit_transform(np.loadtxt(path + "train/Inertial Signals/total_acc_z_train.txt"))

train_t = np.loadtxt(path + "train/y_train.txt").astype(int)
train_s = np.loadtxt(path + "train/subject_train.txt").astype(int)

#test set
test_ax = mmscaler.fit_transform(np.loadtxt(path + "test/Inertial Signals/total_acc_x_test.txt"))
test_ay = mmscaler.fit_transform(np.loadtxt(path + "test/Inertial Signals/total_acc_y_test.txt"))
test_az = mmscaler.fit_transform(np.loadtxt(path + "test/Inertial Signals/total_acc_z_test.txt"))

test_t = np.loadtxt(path + "test/y_test.txt").astype(int)
test_s = np.loadtxt(path + "test/subject_test.txt").astype(int)

print(train_t)

[5 5 5 ... 2 2 2]


In [176]:
train_size = 7352
test_size = 2947
dim_size = 3
sample_size = 128

In [177]:
# trX = (7352, 128, 3) ... trainのX
trX = np.ones((train_size, sample_size, dim_size), float)
print('trX.shape initial:{0}'.format(trX.shape))
for i in range(train_size):
  temp1 = np.ones((dim_size, sample_size), float)
  temp1[0,:] = train_ax[i,:]
  temp1[1,:] = train_ay[i,:]
  temp1[2,:] = train_az[i,:]
  trX[i,:,:] = temp1.reshape(-1,3)
print('trX.shape assigned:{0}'.format(trX.shape))

# t(movement label) or s(subject label) or both ... trainのY (7352,1)
trY = train_t.reshape(-1,1)

# teX = (2947, 3, 128) ... testのX
teX = np.ones((test_size, sample_size, dim_size), float)
print('teX.shape initial:{0}'.format(teX.shape))
for i in range(test_size):
  temp2 = np.ones((dim_size, sample_size), float)
  temp2[0,:] = test_ax[i,:]
  temp2[1,:] = test_ay[i,:]
  temp2[2,:] = test_az[i,:]
  teX[i,:,:] = temp2.reshape(-1,3)
print('trX.shape assigned:{0}'.format(teX.shape))

# testのY 2947行1列
teY = test_t.reshape(-1,1)

trX.shape initial:(7352, 128, 3)
trX.shape assigned:(7352, 128, 3)
teX.shape initial:(2947, 128, 3)
trX.shape assigned:(2947, 128, 3)


In [178]:
datax = np.vstack([trX, teX])
print('datax.shape:{0}'.format(datax.shape))
datay = np.vstack([trY, teY])
print('datay.shape:{0}'.format(datay.shape))
# dataX = trX and teX (10299, 3, 128)
dataX = torch.Tensor(np.array(datax)).to(device)
# dataY = trY and teY (10299,1)
dataY = torch.Tensor(np.array(datay)).to(device)

trainX = torch.Tensor(np.array(trX)).to(device)
trainY = torch.Tensor(np.array(trY)).to(device)

testX = torch.Tensor(np.array(teX)).to(device)
testY = torch.Tensor(np.array(teY)).to(device)

# trainYをone-hotにするためlongにして1次元配列に戻す　-> min~maxを0~5に直す
trainY = trainY.view(-1).long() - 1
print(trainY.shape)
print(torch.min(trainY), torch.max(trainY))

# one-hotにする -> 誤差を計算できるようにfloatに直す
trainY = F.one_hot(trainY, num_classes=-1)
trainY = trainY.float()
print(trainY.shape)
print(trainY)
trY = trainY.cpu().data.numpy()
print(trY.shape)

datax.shape:(10299, 128, 3)
datay.shape:(10299, 1)
torch.Size([7352])
tensor(0, device='cuda:0') tensor(5, device='cuda:0')
torch.Size([7352, 6])
tensor([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        ...,
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], device='cuda:0')
(7352, 6)


## Dataset & Dataloader

In [179]:
class DataSet:
    def __init__(self):
        self.X = trX.astype(np.float32) # 入力
        self.t = trY # 出力

    def __len__(self):
        return len(self.X) # データ数(10)を返す

    def __getitem__(self, index):
        # index番目の入出力ペアを返す
        return self.X[index], self.t[index]

In [180]:
dataset = DataSet()
print('全データ数:',len(dataset))  # 
#print('3番目のデータ:',dataset[3]) 

全データ数: 7352


In [181]:
# さっき作ったDataSetクラスのインスタンスを作成
dataset = DataSet()
# datasetをDataLoaderの引数とすることでミニバッチを作成．
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, \
                                         shuffle=True, drop_last=True)

In [182]:
class MyLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_size = 70
        self.lstm = nn.LSTM(input_size=3, hidden_size=self.hidden_size, num_layers=1, batch_first=True) 
        self.relu = nn.ReLU()
        self.linear = nn.Linear(70, 6)
        self.softmax = nn.Softmax(-1)

    def forward(self, x):
        h_0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)
        c_0 = torch.zeros(1, x.size(0), self.hidden_size).to(device)
        _, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size)
        h_out = self.relu(h_out)
        h_out = self.linear(h_out)
        y_hat = self.softmax(h_out)
        return y_hat


print(summary(MyLSTM(), input_size=(7352, 128, 3), device=torch.device(device)))  

Layer (type:depth-idx)                   Output Shape              Param #
MyLSTM                                   [7352, 6]                 --
├─LSTM: 1-1                              [7352, 128, 70]           21,000
├─ReLU: 1-2                              [7352, 70]                --
├─Linear: 1-3                            [7352, 6]                 426
├─Softmax: 1-4                           [7352, 6]                 --
Total params: 21,426
Trainable params: 21,426
Non-trainable params: 0
Total mult-adds (G): 19.77
Input size (MB): 11.29
Forward/backward pass size (MB): 527.34
Params size (MB): 0.09
Estimated Total Size (MB): 538.72


In [183]:
def train(model, optimizer, X, t):
  model.train()
  y_hat = model(X)
  #print(y_hat)
  # loss = F.mse_loss(y_hat, trainY)
  loss = nn.CrossEntropyLoss()
  output = loss(y_hat, t)
  optimizer.zero_grad()
  output.backward()
  optimizer.step()
  return output.item()

loss = []

def main():
  model = MyLSTM()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  model = model.to(device)

  for epoch in range(500):
    for X, t in dataloader:
      _loss = train(model, optimizer, X.to(device), t.to(device))
      loss.append(_loss)
    if epoch % 20 == 0:
      print(f"Epoch = {epoch+1}, Loss = {_loss:.5f}")
  return model

In [184]:
def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

# create default optimizer for doctests

param_tensor = torch.zeros([1], requires_grad=True)
default_optimizer = torch.optim.SGD([param_tensor], lr=0.1)

# create default trainer for doctests
# as handlers could be attached to the trainer,
# each test must define his own trainer using `.. testsetup:`

def get_default_trainer():

    def train_step(engine, batch):
        return batch

    return Engine(train_step)

# create default model for doctests

# default_model = nn.Sequential(OrderedDict([
#     ('base', nn.Linear(4, 2)),
#     ('fc', nn.Linear(2, 1))
# ]))

# manual_seed(666)

In [185]:
model = main()

Epoch = 1, Loss = 1.62958
Epoch = 21, Loss = 1.54192
Epoch = 41, Loss = 1.12143
Epoch = 61, Loss = 1.10635
Epoch = 81, Loss = 1.07080
Epoch = 101, Loss = 1.11139
Epoch = 121, Loss = 1.09288
Epoch = 141, Loss = 1.11087
Epoch = 161, Loss = 1.18195
Epoch = 181, Loss = 1.05763
Epoch = 201, Loss = 1.06416
Epoch = 221, Loss = 1.13823
Epoch = 241, Loss = 1.10223
Epoch = 261, Loss = 1.09193
Epoch = 281, Loss = 1.51463
Epoch = 301, Loss = 1.39938
Epoch = 321, Loss = 1.50908
Epoch = 341, Loss = 1.59076
Epoch = 361, Loss = 1.29763
Epoch = 381, Loss = 1.24808
Epoch = 401, Loss = 1.34997
Epoch = 421, Loss = 1.27532
Epoch = 441, Loss = 1.27053
Epoch = 461, Loss = 1.59702
Epoch = 481, Loss = 1.65407


In [186]:
def predict(model):
  model.eval()
  train_predict = model(testX)
  print(train_predict.shape)

  data_predict = train_predict.cpu().data.numpy()
  testY_plot = testY.cpu().data.numpy()

  
  data_predict = torch.argmax(train_predict, dim=-1)
  data_predict = F.one_hot(data_predict, num_classes=-1)
  
  metric = ConfusionMatrix(num_classes=6)
  metric.attach(default_evaluator, 'cm')
  y_true = testY.view(-1).int()
  y_pred = data_predict
  print(y_true.shape)
  print(y_pred.shape)

  state = default_evaluator.run([[y_pred, y_true]])
  print(state.metrics['cm'])


In [187]:
predict(model)

torch.Size([2947, 6])
torch.Size([2947])
torch.Size([2947, 6])
tensor([[  0,   0,   0,   0,   0,   0],
        [253,  53, 149,  41,   0,   0],
        [154, 132, 146,  39,   0,   0],
        [159,  35, 192,  34,   0,   0],
        [296,   2,  23, 169,   0,   1],
        [477,  13,   3,  39,   0,   0]])
